In [32]:
#https://www.kaggle.com/anikannal/solar-power-generation-data/tasks

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics import median_absolute_error
from sklearn.linear_model import Ridge
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline

In [2]:
train = pd.read_parquet("train.parquet")
val = pd.read_parquet("val.parquet")

In [3]:
train.head()

index  DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  \
0      0 2020-05-15   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
1  59333 2020-05-15   4135001  z9Y9gH1T5YWrNuG       0.0       0.0   
2  56211 2020-05-15   4135001  wCURE6d3bPkepu2       0.0       0.0   
3  53090 2020-05-15   4135001  uHbuxQJl8lW7ozc       0.0       0.0   
4  49970 2020-05-15   4135001  sjndEbLyjtCKgGv       0.0       0.0   

   DAILY_YIELD  TOTAL_YIELD  AMBIENT_TEMPERATURE  MODULE_TEMPERATURE  \
0          0.0    6259559.0            25.184316           22.857507   
1          0.0    7007866.0            25.184316           22.857507   
2          0.0    6782598.0            25.184316           22.857507   
3          0.0    7038681.0            25.184316           22.857507   
4          0.0    7016832.0            25.184316           22.857507   

   IRRADIATION  15M_YIELD   Y4  Y4WIN  
0          0.0        0.0  0.0    0.0  
1          0.0        0.0  0.0    0.0  
2          0.0        0.0  0.0    0.0  
3          0.0        0.0  0.0    0.0  
4          0.0        0.0  0.0    0.0

# Model Likelihood
- ótima forma de reduzir esparsidade

In [12]:
train2 = train.copy()
train2['WEEKDAY'] = train2['DATE_TIME'].dt.weekday
train2['HOUR'] = train2['DATE_TIME'].dt.hour
train2['MINUTE'] = train2['DATE_TIME'].dt.minute


val2 = val.copy()
val2['WEEKDAY'] = val2['DATE_TIME'].dt.weekday
val2['HOUR'] = val2['DATE_TIME'].dt.hour
val2['MINUTE'] = val2['DATE_TIME'].dt.minute

cats = ['SOURCE_KEY', 'WEEKDAY','HOUR', 'MINUTE']

new_features = set()
new_features_models = list()
for c in cats:
    
    mdl = make_pipeline(OneHotEncoder(cols=[c]), Ridge())
    mdl.fit(train2[[c]], train2['Y4WIN'])
    feature = mdl.predict(train2[[c]])
    
    train2["P_{}".format(c)] = mdl.predict(train2[[c]])
    val2["P_{}".format(c)] = mdl.predict(val2[[c]])
    
    new_features.add("P_{}".format(c))
    
    #new_features_models.append(mdl)

/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif

In [5]:
features = list(new_features)

from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

Xtr, ytr = train2[features].fillna(-1), train2['Y4']
mdl.fit(Xtr,ytr)

p = mdl.predict(val2[features].fillna(-1))
median_absolute_error(val2['Y4'], p)

3.0736272280950985

# Reduzir cardinalidade

In [15]:
level_counts = train['SOURCE_KEY'].value_counts()
level_counts

bvBOhCH3iADSZry    1526
1BY6WEcLGh8j5v7    1525
7JYdWkrLSPkdwr4    1511
VHMLBKoKgIrUVDU    1511
ih0vzX44oOqAx2f    1508
ZnxXDlPa8U1GXgE    1508
pkci93gMrogZuBj    1506
z9Y9gH1T5YWrNuG    1506
iCRJl6heRkivqQ3    1506
wCURE6d3bPkepu2    1506
uHbuxQJl8lW7ozc    1506
McdE0feGgRqW7Ca    1505
sjndEbLyjtCKgGv    1505
zVJPv84UY57bAof    1505
rGa61gmuvPhdLxV    1505
ZoEaEvLYb1n2sOq    1504
zBIq5rxdHJRwDNY    1500
1IF53ai7Xc0U56Y    1496
3PZuoBAID5Wc2HD    1496
WRmjgnKYAwPKWDb    1496
adLQvlD726eNBSB    1496
YxYtjZvoooNbGkE    1485
Name: SOURCE_KEY, dtype: int64

In [7]:
low_count = set(level_counts[level_counts < 1500].index)
low_count

{'1IF53ai7Xc0U56Y',
 '3PZuoBAID5Wc2HD',
 'WRmjgnKYAwPKWDb',
 'YxYtjZvoooNbGkE',
 'adLQvlD726eNBSB'}

In [19]:
train2 = train.copy()
train2['WEEKDAY'] = train2['DATE_TIME'].dt.weekday
train2['HOUR'] = train2['DATE_TIME'].dt.hour
train2['MINUTE'] = train2['DATE_TIME'].dt.minute

train2['SOURCE_KEY'] = train2['SOURCE_KEY'].map(lambda x: "LOW_COUNT" if x in low_count else x)


val2 = val.copy()
val2['WEEKDAY'] = val2['DATE_TIME'].dt.weekday
val2['HOUR'] = val2['DATE_TIME'].dt.hour
val2['MINUTE'] = val2['DATE_TIME'].dt.minute

val2['SOURCE_KEY'] = val2['SOURCE_KEY'].map(lambda x: "LOW_COUNT" if x in low_count else x)

cats = ['SOURCE_KEY', 'WEEKDAY','HOUR', 'MINUTE']

new_features = set()
new_features_models = list()
for c in cats:
    
    mdl = make_pipeline(OneHotEncoder(cols=[c]), Ridge())
    mdl.fit(train2[[c]], train2['Y4WIN'])
    feature = mdl.predict(train2[[c]])
    
    train2["P_{}".format(c)] = mdl.predict(train2[[c]])
    val2["P_{}".format(c)] = mdl.predict(val2[[c]])
    
    new_features.add("P_{}".format(c))
    
    #new_features_models.append(mdl)

/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif

In [22]:
features = list(new_features)

from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

Xtr, ytr = train2[features].fillna(-1), train2['Y4WIN']
mdl.fit(Xtr,ytr)

p = mdl.predict(val2[features].fillna(-1))
median_absolute_error(val2['Y4'], p)

2.646281609412752

# PCA (e derivados)

In [27]:
from sklearn.decomposition import PCA
train2 = train.copy()
train2['WEEKDAY'] = train2['DATE_TIME'].dt.weekday
train2['HOUR'] = train2['DATE_TIME'].dt.hour
train2['MINUTE'] = train2['DATE_TIME'].dt.minute


val2 = val.copy()
val2['WEEKDAY'] = val2['DATE_TIME'].dt.weekday
val2['HOUR'] = val2['DATE_TIME'].dt.hour
val2['MINUTE'] = val2['DATE_TIME'].dt.minute

cats = ['SOURCE_KEY', 'WEEKDAY','HOUR', 'MINUTE']

new_features = set()
new_features_models = list()
for c in cats:
    
    mdl = make_pipeline(OneHotEncoder(cols=[c]), PCA(n_components=1))
    mdl.fit(train2[[c]])
    feature = mdl.transform(train2[[c]])
    
    train2["P_{}".format(c)] = mdl.transform(train2[[c]])
    val2["P_{}".format(c)] = mdl.transform(val2[[c]])
    
    new_features.add("P_{}".format(c))
    
    #new_features_models.append(mdl)

/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif

In [28]:
features = list(new_features)

from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

Xtr, ytr = train2[features].fillna(-1), train2['Y4WIN']
mdl.fit(Xtr,ytr)

p = mdl.predict(val2[features].fillna(-1))
median_absolute_error(val2['Y4'], p)

2.632892334408959